In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# hyper params 
block_size = 8
batch_size = 4

max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2 # prevents over fitting 

cuda


In [30]:
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [31]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)



In [32]:
n = int(0.8*len(data))
train_data = data[:n]
val_data=data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
   # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')

print('inputs:')
print(x)

print('targets: ')
print(y)


inputs:
tensor([[62, 68, 67,  1, 68, 59,  1, 58],
        [76, 53,  1, 61, 58,  1, 62, 72],
        [ 1, 54, 67, 57,  1, 67, 68,  1],
        [56, 58,  1, 68, 59,  1, 73, 61]], device='cuda:0')
targets: 
tensor([[68, 67,  1, 68, 59,  1, 58, 62],
        [53,  1, 61, 58,  1, 62, 72,  9],
        [54, 67, 57,  1, 67, 68,  1, 66],
        [58,  1, 68, 59,  1, 73, 61, 58]], device='cuda:0')


In [33]:
block_size = 8 

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)
    

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [39]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()
    return out

In [34]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        ''' This is for the behind the scenes shit helping us understand what is going on under the hood --> also a lot easier to debug
        What are the logits:
        a bunch of normaized floating point numbers
        we sum the numbers and then div each number in the set buy the total (normalization) -> this gives us a prob dist of what we want to predict
        '''        
        logits = self.token_embeding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # Batch, Time, Channels
    # What does view do: al;lwos us to unpack with.shapoe and then pack back totgether
            logits = logits.view(B*T, C) #The batch and time arnt suepr important so we can blend them together
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # we use veiw to ensure that the shape that this funct expects is met -> it exprect b by c bt t 
            

        return logits, loss

    def generate(self, index, max_new_tokens):

        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes b, c
            # apply softmax tro get probs
            probs = F.softmax(logits, dim=-1) # b,c
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # b, 1
            # append the sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # B, T+1
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



N)bI_ujz)GQDeGB:N2E48v3iWfN&-MZkezS_u8rzz
DhbPKrpz)9A;gG39&﻿dpCu3lCZwIVk544teMteRc_QS0d['U.2e!;q9teW:*VA4,2wk(8CKn_4HZjL?aGDc:7 kNoPFQ)kMmi
(I3v
,Y-bBMAR3lQPx7bQ,lSemgDgTzSM8t-c2?JOwx.k48f0yzl&dcegue7 qTaSi4qBpPO7,xISn'S3Zy0a'Q PN&8IH.C-1q2f0r!E0ezfgH6?([V-DGyiNh1UACu*5]e"4N&ZaK8XX﻿Co-*HxCI(5Bo;"FPU9oIvdndXBg
Ci?XbO_epid,Q*Z,ZOin'wJ﻿6yZt6i?"0KGuC7!eSOxVZhku﻿.ZNkrM]'ZSz:n'Ti6Gtedn﻿mh)PRm&8XQ.N﻿6yzhbwf7i9GC﻿uv'jR9"zIUGp2k:9a)7C:NZvZ*3yYcfBo4mstJx
:pvPp;o-G]n',cPS?443l4ydDj?frrCZYh]Ie7v&﻿*VJmE;)9lW


In [46]:
# create a pytorch optimiser

optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f" step {iter}, train Loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimiser.zero_grad(set_to_none=True)

    loss.backward()
    optimiser.step()

print(loss.item())

 step 0, train Loss: 2.5062, val loss: 2.5157
 step 250, train Loss: 2.4789, val loss: 2.5030
 step 500, train Loss: 2.4769, val loss: 2.5163
 step 750, train Loss: 2.4794, val loss: 2.4933
 step 1000, train Loss: 2.4462, val loss: 2.4931
 step 1250, train Loss: 2.4784, val loss: 2.5015
 step 1500, train Loss: 2.4684, val loss: 2.4969
 step 1750, train Loss: 2.4840, val loss: 2.5131
 step 2000, train Loss: 2.4732, val loss: 2.4931
 step 2250, train Loss: 2.4685, val loss: 2.5298
 step 2500, train Loss: 2.4689, val loss: 2.5162
 step 2750, train Loss: 2.4674, val loss: 2.5092
 step 3000, train Loss: 2.4847, val loss: 2.4950
 step 3250, train Loss: 2.4366, val loss: 2.5052
 step 3500, train Loss: 2.4725, val loss: 2.5111
 step 3750, train Loss: 2.4614, val loss: 2.5243
 step 4000, train Loss: 2.4488, val loss: 2.5435
 step 4250, train Loss: 2.4435, val loss: 2.4918
 step 4500, train Loss: 2.4432, val loss: 2.5214
 step 4750, train Loss: 2.4571, val loss: 2.4840
 step 5000, train Loss: 2.

In [45]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


s, ss the oth thes ad.

omocopuble te t wat he ftowize he sor pilie fal


"thald aded oug ws che, we char tllprousider tttisweel.
B)Cs?"YFaghe rey,"N'lye thesthe t, one, ale thee fo q2;je an
"Huben g d pomancang an theait fonveve t  y th't

"w;8_ist
mecef icecht warfre fulonge san d oueimed y
Arag d mug, al, f my gr!PZH]f me

and t guthenepipurza Ohe thea en aifoood m dser f d wo wh animabesein w an-rot the way torand rd tlismb," waly, lly eningh west-;AI've ntherrct ndiren w
Cr we tledar ifre c
